In [8]:
import pandas as pd

import os

os.environ["OMP_NUM_THREADS"] = "2"
os.environ["OPENBLAS_NUM_THREADS"] = "2"
os.environ["MKL_NUM_THREADS"] = "2"
os.environ["VECLIB_MAXIMUM_THREADS"] = "2"
os.environ["NUMEXPR_NUM_THREADS"] = "2"



import pandas as pd

# Load and combine datasets (as before)
datasets = {
    'benign': pd.read_csv('labeled_dataset_benign.csv'),
    'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
    'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')
}
combined_df = pd.concat([datasets['benign'], datasets['dns'], datasets['mitm']], ignore_index=True)
combined_df = combined_df.drop('dst_mac', axis=1)

print(combined_df.shape)

C:\Users\nazih\AppData\Local\Temp\ipykernel_19176\1938280762.py:18: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'dns': pd.read_csv('labeled_dataset_DNS_Spoofing.csv'),
C:\Users\nazih\AppData\Local\Temp\ipykernel_19176\1938280762.py:19: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  'mitm': pd.read_csv('labeled_dataset_MITM_ArpSpoofing.csv')


(883525, 135)


In [9]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

def is_hex_mac(mac):
    if pd.isna(mac) or mac == 'unknown':
        return False
    pattern = r'^([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})$'
    return bool(re.match(pattern, str(mac)))


# Remove rows where src_mac is a real MAC address
combined_df = combined_df[~combined_df['src_mac'].apply(is_hex_mac)]

# Drop duplicates
print(f"Number of duplicate rows: {combined_df.duplicated().sum()}")
combined_df = combined_df.drop_duplicates()
print(f"New shape after removing duplicates: {combined_df.shape}")

# Identify categorical columns
categorical_cols = [col for col in combined_df.columns if combined_df[col].nunique() < 10 or combined_df[col].dtype == 'object']
print(f"Categorical features: {categorical_cols}")

# Convert all categorical columns to string and fill missing with 'unknown'
for col in categorical_cols:
    combined_df[col] = combined_df[col].astype(str).fillna('unknown')

# Fill missing numerical values with median
numerical_cols = combined_df.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_cols:
    combined_df[col] = combined_df[col].fillna(combined_df[col].median())

# --- Step 2: Split target labels and features ---
X = combined_df.drop(['Label', 'src_mac'], axis=1, errors='ignore')
y_attack = combined_df['Label']
y_device = combined_df['src_mac']

# Encode device labels
le_device = LabelEncoder()
y_device_encoded = le_device.fit_transform(y_device)

from sklearn.preprocessing import MultiLabelBinarizer

# 1. Create a column that holds both relevant labels as a list for each row
# This creates a list ['Attack_Name', 'Device_MAC'] for every packet
combined_df['All_Labels'] = combined_df.apply(
    lambda row: [row['Label'], row['src_mac']], axis=1
)

# 2. Initialize and fit the binarizer
mlb = MultiLabelBinarizer()
Y_multilabel = mlb.fit_transform(combined_df['All_Labels'])

# 3. Store the column names (needed for evaluation later)
multilabel_class_names = mlb.classes_


# Keep only numeric columns
X_numeric = X.select_dtypes(include=['number'])

# Normalize features using StandardScaler (zero mean, unit variance)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)

# Apply PCA to retain 95% variance
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_scaled)

pca_feature_names = [f'PC{i+1}' for i in range(X_reduced.shape[1])]

# Convert to DataFrame for ease of use
X_reduced_df = pd.DataFrame(X_reduced, columns=pca_feature_names, index=X.index)

# Replace the single-label splits with the multi-label split:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_reduced_df,       # Your PCA features
    Y_multilabel,       # Your multi-label target matrix
    test_size=0.2, 
    random_state=42
)


Number of duplicate rows: 0
New shape after removing duplicates: (432555, 135)
Categorical features: ['src_mac', 'src_ip', 'dst_ip', 'port_class_dst', 'l4_tcp', 'l4_udp', 'ttl', 'handshake_version', 'handshake_ciphersuites', 'tls_server', 'http_request_method', 'http_host', 'http_response_code', 'user_agent', 'dns_server', 'dns_query_type', 'dns_len_ans', 'device_mac', 'eth_src_oui', 'eth_dst_oui', 'highest_layer', 'http_uri', 'http_content_len', 'http_content_type', 'icmp_type', 'icmp_checksum_status', 'icmp_data_size', 'Label']


C:\Users\nazih\AppData\Local\Temp\ipykernel_19176\2122749699.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df['All_Labels'] = combined_df.apply(


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
import lightgbm as lgb
from sklearn.metrics import accuracy_score, jaccard_score, hamming_loss, classification_report

# --- 1. Define the Base Estimator (LightGBM) ---
print("--- Training Multi-Label LightGBM ---")
# Apply regularization and weighted loss to the base model
lgb_base = lgb.LGBMClassifier(
    n_estimators=100, 
    random_state=42, 
    n_jobs=2,
    max_depth=8,              # Regularization: Limit tree depth
    reg_alpha=0.2,            # Regularization: L1
    # Weighted Loss: Applies balancing to each of the output's binary problems
    class_weight='balanced'   
)

# --- 2. Wrap it with MultiOutputClassifier ---
# This trains M LightGBM models (where M is the number of total unique labels)
lgb_multioutput = MultiOutputClassifier(lgb_base, n_jobs=-1)

# --- 3. Fit the Model ---
# This will take longer than a single multi-class model
lgb_multioutput.fit(X_train, Y_train)

# --- 4. Prediction ---
Y_pred_lgbm = lgb_multioutput.predict(X_test)

--- Training Multi-Label LightGBM ---


In [ ]:
def evaluate_multilabel_model(Y_test, Y_pred, class_names):
    """Calculates and prints key multi-label metrics."""
    print("\n======== Evaluation: LightGBM Multi-Output ========")
    
    # 1. Subset Accuracy (STRICTEST: Requires *ALL* labels to match exactly)
    subset_acc = accuracy_score(Y_test, Y_pred)
    print(f"Subset Accuracy (Exact Match): {subset_acc:.4f}")

    # 2. Jaccard Score (Average of Intersection over Union)
    # This is often the most informative metric for multi-label
    jaccard = jaccard_score(Y_test, Y_pred, average='samples')
    print(f"Jaccard Score (Label Similarity): {jaccard:.4f}")

    # 3. Hamming Loss (Fraction of incorrect labels)
    # Lower is better; a perfect score is 0.0
    hamming = hamming_loss(Y_test, Y_pred)
    print(f"Hamming Loss (Error Rate): {hamming:.4f}")
    
    # 4. Detailed Per-Label Metrics (Micro/Macro averages)
    print("\n--- Micro-Averaged Metrics (Focus on Total Correctness) ---")
    # Note: Target names should come from the mlb.classes_ variable
    print(classification_report(Y_test, Y_pred, 
                                target_names=class_names, 
                                zero_division=0, 
                                output_dict=False))

# Run the evaluation
evaluate_multilabel_model(Y_test, Y_pred_lgbm, multilabel_class_names)